In [ ]:
import requests
import json
from elasticsearch import Elasticsearch, helpers

# Define the local file path
LOCAL_JSON_FILE = 'documents.json'
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'

# Attempt to load from local file, otherwise download and save
try:
    with open(LOCAL_JSON_FILE, 'r', encoding='utf-8') as f:
        documents_raw = json.load(f)
    print("Loaded documents from local file.")
except FileNotFoundError:
    print("Local file not found. Downloading...")
    docs_response = requests.get(docs_url)
    docs_response.raise_for_status()
    documents_raw = docs_response.json()
    with open(LOCAL_JSON_FILE, 'w', encoding='utf-8') as f:
        json.dump(documents_raw, f, indent=4)
    print("Downloaded and saved documents locally.")

# Process the documents
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print("\nFirst document after processing:")
print(documents[0])

In [3]:
# 2. Connect to Elasticsearch
# Replace with your Elasticsearch host and port if different
es = Elasticsearch("http://localhost:9200")

# Check if connection is successful
if not es.ping():
    raise ValueError("Connection to Elasticsearch failed!")
else:
    print("Successfully connected to Elasticsearch!")

Successfully connected to Elasticsearch!


In [4]:
# 3. Define your index name
index_name = "llm_zoomcamp_documents"

# 4. Optional: Define a mapping for your index
# This helps Elasticsearch understand your data types and how to index them.
# If you don't define this, Elasticsearch will guess.
mappings = {
    "properties": {
        "text": {"type": "text"},
        "section": {"type": "text"}, # Use keyword for exact matches (e.g., filtering)
        "question": {"type": "text"},
        "course": {"type": "keyword"}
    }
}

# 5. Create the index (if it doesn't exist) with the mapping
if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting and recreating...")
    es.indices.delete(index=index_name) # Be careful with this in production!

es.indices.create(index=index_name, body={"mappings": mappings})
print(f"Index '{index_name}' created successfully.")

Index 'llm_zoomcamp_documents' created successfully.


## Q2

it should be .index to create index, but we can use bulk index

In [6]:
# 6. Prepare documents for bulk indexing
# The helpers.bulk function expects a list of dictionaries,
# where each dictionary has at least '_index' and '_source' keys.
actions = [
    {
        "_index": index_name,
        "_source": doc
    }
    for doc in documents
]

# 7. Perform bulk indexing
try:
    success, failed = helpers.bulk(es, actions, index=index_name)
    print(f"Successfully indexed {success} documents.")
    if failed:
        print(f"Failed to index {len(failed)} documents. First 5 failures: {failed[:5]}")
except Exception as e:
    print(f"An error occurred during bulk indexing: {e}")

Successfully indexed 948 documents.


In [8]:
# 8. Verify the count of documents
doc_count = es.count(index=index_name)['count']
print(f"Total documents in '{index_name}': {doc_count}")

Total documents in 'llm_zoomcamp_documents': 948


## Q3

In [9]:
# --- Search Query ---
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5, # We only need the top result
    "query": {
        "multi_match": {
            "query": query,
            "fields": ["question^4", "text"], # question field gets a boost of 4
            "type": "best_fields"
        }
    }
}

response = es.search(index=index_name, body=search_query)

# Extract and print the score of the top result
if response['hits']['hits']:
    top_hit = response['hits']['hits'][0]
    score = top_hit['_score']
    print(f"\nTop ranking result:")
    print(f"Score: {score}")
    print(f"Question: {top_hit['_source'].get('question')}")
    print(f"Text: {top_hit['_source'].get('text')}")
    print(f"Course: {top_hit['_source'].get('course')}")
else:
    print("No results found.")


Top ranking result:
Score: 44.50556
Question: How do I debug a docker container?
Text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)
Course: machine-learning-zoomcamp


In [ ]:
result_docs = []

for hit in response['hits']['hits']:
    print(hit['_score'], hit['_source'])

44.50556 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
35.433445 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', 'course': 'machine-learning-zoomcamp'}
33.70974 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"

## Q4

In [14]:
# --- NEW SEARCH QUERY ---
query = "How do copy a file to a Docker container?"
course_filter = "machine-learning-zoomcamp"
num_results = 3

search_query = {
    "size": num_results,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course_filter
                }
            }
        }
    }
}

print(f"\nSearching for: '{query}' in course '{course_filter}' (top {num_results} results):")
response = es.search(index=index_name, body=search_query)

# Print the top results and their scores
for hit in response['hits']['hits']:
    print(f"Course: {hit['_source']['course']}, Question: {hit['_source']['question']}, Score: {hit['_score']}")


Searching for: 'How do copy a file to a Docker container?' in course 'machine-learning-zoomcamp' (top 3 results):
Course: machine-learning-zoomcamp, Question: How do I debug a docker container?, Score: 73.38676
Course: machine-learning-zoomcamp, Question: How do I copy files from my local machine to docker container?, Score: 66.688705
Course: machine-learning-zoomcamp, Question: How do I copy files from a different folder into docker container’s working directory?, Score: 59.812744


## Q5

In [16]:
# 1. Build the context string
context_template = """
Q: {question}
A: {text}
""".strip()

context_entries = []
for hit in response['hits']['hits']:
    doc = hit['_source']
    context_entries.append(context_template.format(question=doc['question'], text=doc['text']))

context = "\n\n".join(context_entries)

# 2. Define the main question
question_to_llm = "How do I execute a command in a running docker container?"

# 3. Construct the final prompt
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

final_prompt = prompt_template.format(question=question_to_llm, context=context)

print("\n--- Generated Prompt for LLM ---")
print(final_prompt)
print("\n--- End of Prompt ---")


--- Generated Prompt for LLM ---
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/loc

In [17]:
len(final_prompt)

1462

In [19]:
import tiktoken
# Calculate tokens
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(final_prompt)
num_tokens = len(tokens)

print(f"The prompt has {num_tokens} tokens.")

The prompt has 322 tokens.


In [20]:
tokens

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 734,
 48,
 25,
 3253,
 621,
 357,
 15199,
 261,
 62275,
 9282,
 3901,
 32,
 25,
 41281,
 290,
 9282,
 3621,
 306,
 25383,
 6766,
 326,
 151187,
 290,
 7251,
 4859,
 11,
 813,
 484,
 480,
 13217,
 261,
 38615,
 6348,
 558,
 68923,
 2461,
 533,
 278,
 2230,
 7962,
 4859,
 38615,
 464,
 3365,
 523,
 3335,
 290,
 9282,
 382,
 4279,
 6788,
 11,
 15792,
 261,
 6348,
 306,
 290,
 4857,
 9282,
 734,
 68923,
 10942,
 350,
 6555,
 290,
 9282,
 26240,
 446,
 68923,
 25398,
 533,
 278,
 464,
 6896,
 26240,
 29,
 38615,
 198,
 6103,
 277,
 10732,
 391,
 79771,
 1029,
 48,
 25,
 3253,
 621,
 357,
 5150,
 6291,
 591,
 922,
 2698,
 7342,
 316,
 62275,
 9282,
 3901,
 32,
 2

In [22]:
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.2:1b",
  messages=[
            {"role": "user", "content": final_prompt}
        ],
        temperature=0.0, # Often good to keep low for factual retrieval
)
display(Markdown((response.choices[0].message.content)))

Based on the provided CONTEXT, I will answer your question:

To execute a command in a running Docker container, you can use the following methods:

1. Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
2. Execute a command in the specific container by using `docker ps` to find the container-id and then `docker exec -it <container-id> bash`.

This will allow you to interact with the container as if it were your local machine.